In [5]:
import os
import csv
from itertools import product
from pydub import AudioSegment

In [6]:
# Path to the parent folder containing the song folders
# songs_folder = "Songs"
songs_folder = "Songs" #Modify the path and the names of the audio files. 

# Get all song folders dynamically
song_folders = [os.path.join(songs_folder, folder) for folder in os.listdir(songs_folder) if os.path.isdir(os.path.join(songs_folder, folder))]

# List of all 15 audio file names (common across all songs)
audio_file_names = [
    "A_Melody1.wav", "B_Melody2.wav", "C_Melody3.wav",
    "E_Harmony1.wav", "F_Harmony2.wav",
    "H_AddVoice1.wav", "I_AddVoice2.wav", "J_AddVoice3.wav",
    "L_Bass1.wav", "M_Bass2.wav", # Modify the names of the audio files based on file names
    "O_Drum1.wav", "P_Drum2.wav",
    "Q_Ambient1.wav", "R_Ambient2.wav", "S_Ambient3.wav"
]

# Output folder for all combinations
output_folder = "Combined_Songs"
os.makedirs(output_folder, exist_ok=True)

# CSV file to log all combinations
csv_file = os.path.join(output_folder, "combinations_log.csv")

In [7]:

# Function to save the combined audio for a given combination
def save_combination(combination, output_path):
    combined_audio = None  # Start with None
    for audio_file in combination:
        if audio_file and os.path.isfile(audio_file):  # Skip empty or invalid paths
            try:
                audio = AudioSegment.from_wav(audio_file)
                if combined_audio is None:
                    combined_audio = audio  # Initialize with the first audio
                else:
                    combined_audio = combined_audio.overlay(audio)  # Overlay audio
            except Exception as e:
                print(f"Error processing file {audio_file}: {e}")
        else:
            print(f"Skipping invalid path: {audio_file}")
    
    if combined_audio:  # Only export if valid audio was combined
        combined_audio.export(output_path, format="wav")
        print(f"Exported: {output_path}")
    else:
        print(f"Skipping empty combination for {output_path}")

In [8]:
def generate_group_combinations(group_files):
    """
    Given a list of files belonging to one group,
    return all combinations where either:
    - no file is active (OFF), or
    - exactly one file is active (ON).
    """
    combinations = [()]  # The empty tuple means "OFF"
    for file in group_files:
        combinations.append((file,))
    return combinations

In [ ]:
# Write combinations to a CSV and generate files
with open(csv_file, mode="w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header
    header = ["Song", "Combination Number"] + audio_file_names + ["Output File"]
    csv_writer.writerow(header)

    # Process each song folder
    for song_folder in song_folders:
        song_name = os.path.basename(song_folder)
        print(f"Processing song: {song_name}")

        # Create a separate folder for combined songs for this song
        song_output_folder = os.path.join(output_folder, f"{song_name}_combined_songs")
        os.makedirs(song_output_folder, exist_ok=True)

        # Create paths for all 15 audio files in this folder
        audio_files = [os.path.join(song_folder, file) for file in audio_file_names]

        # Generate valid combinations for all groups
        all_combinations = list(product(
            generate_group_combinations(audio_files[:3]),  # Melody
            generate_group_combinations(audio_files[3:5]),  # Harmony
            generate_group_combinations(audio_files[5:8]),  # AddVoice
            generate_group_combinations(audio_files[8:10]),  # BassLine
            generate_group_combinations(audio_files[10:11]),  # Percussion
            generate_group_combinations(audio_files[11:12]),  # BassDrum
            generate_group_combinations(audio_files[12:])  # Ambient
        ))

        # Filter out the empty combination (where all groups are "none")
        valid_combinations = [
            combination for combination in all_combinations if any(combination)
        ]

        # Print the total number of valid combinations
        print(f"Total valid combinations for {song_name}: {len(valid_combinations)}")

        # Process each combination
        for i, combination in enumerate(valid_combinations, start=1):
            # Determine ON/OFF for each audio file
            row = [song_name, i]  # Start with the song name and combination number
            for audio_file_name in audio_file_names:
                # Check if the current file is in the combination
                row.append("ON" if any(audio_file_name in file for file in combination if file) else "OFF")

            # Generate output file name
            output_file = os.path.join(song_output_folder, f"{song_name}_combination_{i}.wav")
            row.append(output_file)  # Add the output file path

            # Save the combination audio file
            save_combination(
                [file for group in combination for file in group if file and os.path.isfile(file)],
                output_file
            )

            # Write the row to the CSV
            csv_writer.writerow(row)

print(f"Processed all songs and logged combinations to {csv_file}.")